In [1]:
pkg"free MathOpt"

LoadError: UndefVarError: @pkg_str not defined

In [1]:
using Revise

In [19]:
using SumOfSquares
function CondensedMatterSOS.energy(H, maxdegree, solver;
    cone=NonnegPolyInnerCone{SumOfSquares.COI.HermitianPositiveSemidefiniteConeTriangle}(),
    kws...
)
    model = MOI.instantiate(solver, with_bridge_type=Float64)
    MOI.Bridges.add_bridge(model, SumOfSquares.Bridges.Constraint.SOSPolynomialBridge{Complex{Float64}})
    MOI.Bridges.add_bridge(model, SumOfSquares.Bridges.Constraint.EmptyBridge{Float64})
    MOI.Bridges.add_bridge(model, SumOfSquares.Bridges.Constraint.PositiveSemidefinite2x2Bridge{Float64})
    MOI.Bridges.add_bridge(model, PolyJuMP.ZeroPolynomialBridge{Complex{Float64}})
    MOI.Bridges.add_bridge(model, SumOfSquares.COI.Bridges.Variable.HermitianToSymmetricPSDBridge{Float64})
    MOI.Bridges.add_bridge(model, SumOfSquares.COI.Bridges.Constraint.SplitZeroBridge{Float64})
    return model
    γ = MOI.SingleVariable(MOI.add_variable(model))
    cert = SumOfSquares.Certificate.MaxDegree(cone, MonomialBasis, maxdegree)
    @show H
    @show γ
    @show H - (1.0 + 0.0im) * γ
    #cert = SumOfSquares.Certificate.SparseIdeal(MonomialSparsity(1), cert)
    c = SumOfSquares.add_constraint(model, H - (1.0 + 0.0im) * γ, SOSCone(); ideal_certificate=cert, kws...)
    MOI.set(model, MOI.ObjectiveSense(), MOI.MAX_SENSE)
    MOI.set(model, MOI.ObjectiveFunction{MOI.SingleVariable}(), γ)
    MOI.optimize!(model)
    @show MOI.Bridges.bridge(model, c).gram_basis
    @show MOI.Bridges.bridge(model, c).gram_basis
    #print(Base.stdout, model.model, variable_name = MOI.name_or_default_name)
    MosekTools.Mosek.writedata(model.model.optimizer.task, "unbounded.cbf")
    if MOI.get(model, MOI.TerminationStatus()) != MOI.OPTIMAL
        @warn("Termination status: $(MOI.get(model, MOI.TerminationStatus())), $(MOI.get(model, MOI.RawStatusString()))")
    end
    ν = MOI.get(model, SumOfSquares.GramMatrixAttribute(), c)
    MOI.get(model, MOI.ObjectiveValue()), ν
end

In [ ]:
using CondensedMatterSOS
@spin σ[1:2]
function hamiltonian(σ)
    σx, σy, σz = σ
    N = length(σx)
    return -sum(σx[n] * σx[n+1] for n in 1:(N-1)) - σx[N] * σx[1] - sum(σz)
end
hamiltonian(σ)

┌ Info: Precompiling CondensedMatterSOS [676dea7e-7535-42cf-a5c4-14732a6b7aba]
└ @ Base loading.jl:1260
┌ Warning: Package CondensedMatterSOS does not have Reexport in its dependencies:
│ - If you have CondensedMatterSOS checked out for development and have
│   added Reexport as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with CondensedMatterSOS
└ Loading Reexport into CondensedMatterSOS from project dependency, future warnings for CondensedMatterSOS are suppressed.


In [9]:
function hamiltonian_energy(N, maxdegree, solver; kws...)
    @spin σ[1:N]
    H = 1.0 * hamiltonian(σ)
    energy(H, maxdegree, solver; kws...)
end

hamiltonian_energy (generic function with 1 method)

In [ ]:
( im x[-36] - x[-12] - im x[-36] - x[-12] - 2)σˣ₁σˣ₂ +
(-im x[-40] + x[-17] + im x[-40] + x[-17])σˣ₁σʸ₂ +
(-im x[-45] + x[-23] + im x[-45] + x[-23])σˣ₁σᶻ₂ +
(-im x[-37] + x[-13] + im x[-37] + x[-13])σʸ₁σˣ₂ +
(-im x[-41] + x[-18] + im x[-41] + x[-18])σʸ₁σʸ₂ +
(-im x[-46] + x[-24] + im x[-46] + x[-24])σʸ₁σᶻ₂ +
(-im x[-38] + x[-14] + im x[-38] + x[-14])σᶻ₁σˣ₂ +
(-im x[-42] + x[-19] + im x[-42] + x[-19])σᶻ₁σʸ₂ +
(-im x[-47] + x[-25] + im x[-47] + x[-25])σᶻ₁σᶻ₂ +
(-x[-34] - im x[-9] - x[-34] + im x[-9] - im x[-29] + x[-2] + im x[-29] + x[-2])σˣ₁ +
( x[-33] + im x[-8] - im x[-30] + x[-4] + x[-33] - im x[-8] + im x[-30] + x[-4])σʸ₁ +
(im x[-32] - x[-7] + x[-31] + im x[-5] + x[-31] - im x[-5] - im x[-32] - x[-7] + 1)σᶻ₁ +
(-x[-49] - im x[-27] - x[-49] + im x[-27] - im x[-35] + x[-11] + im x[-35] + x[-11])σˣ₂ +
(x[-48] + im x[-26] - im x[-39] + x[-16] + x[-48] - im x[-26] + im x[-39] + x[-16])σʸ₂ +
(im x[-44] - x[-22] + x[-43] + im x[-20] + x[-43] - im x[-20] - im x[-44] - x[-22] + 1)σᶻ₂ +
(x[1] - x[-28] - x[-21] - x[-15] - x[-10] - x[-6] - x[-3] - x[-1])

In [20]:
using CSDP
using JuMP
solver = optimizer_with_attributes(
    () -> MOIU.CachingOptimizer(MOIU.UniversalFallback(MOIU.Model{Float64}()), CSDP.Optimizer()),
    MOI.Silent() => true
)
model = hamiltonian_energy(2, 2, solver)

MOIB.LazyBridgeOptimizer{MOIU.CachingOptimizer{CSDP.Optimizer,MOIU.UniversalFallback{MOIU.Model{Float64}}}}
with 0 variable bridges
with 0 constraint bridges
with 0 objective bridges
with inner model MOIU.CachingOptimizer{CSDP.Optimizer,MOIU.UniversalFallback{MOIU.Model{Float64}}}
  in state ATTACHED_OPTIMIZER
  in mode AUTOMATIC
  with model cache MOIU.UniversalFallback{MOIU.Model{Float64}}
    with 1 optimizer attribute
    fallback for MOIU.Model{Float64}
  with optimizer CSDP.Optimizer

In [31]:
MOI.Bridges.supports_add_constrained_variables(b.model, MOI.Reals)

UndefVarError: UndefVarError: supports_add_constrained_variables not defined

In [ ]:
## using CSDP
using JuMP
solver = optimizer_with_attributes(
    () -> MOIU.CachingOptimizer(MOIU.UniversalFallback(MOIU.Model{Float64}()), CSDP.Optimizer()),
    MOI.Silent() => true
)
bound, ν = hamiltonian_energy(2, 2, solver)
bound

In [ ]:
MOI.Bridges.is_bridged(b::MOI.Bridges.AbstractBridgeOptimizer, vi::MOI.VariableIndex) = vi.value < 0 && MOI.Bridges.Variable.has_bridges(MOI.Bridges.Variable.bridges(b))

In [ ]:
using SumOfSquares
Base.:+(a::Int, f::MOI.ScalarAffineFunction{Complex{Float64}}) = convert(Complex{Float64}, a) + f
Base.:+(a::Complex{Float64}, f::MOI.ScalarAffineFunction{Complex{Float64}}) = MOI.Utilities.operate(+, Complex{Float64}, a, f)
function MOI.Utilities.operate(::typeof(*), ::Type{Complex{Float64}}, a::Complex{Float64}, f::MOI.ScalarAffineFunction{Float64})
    MOI.Utilities.operate(*, Complex{Float64}, a, convert(MOI.ScalarAffineFunction{Complex{Float64}}, f))
end
function Base.convert(::Type{MOI.ScalarAffineFunction{T}}, f::MOI.ScalarAffineFunction) where T
    return MOI.ScalarAffineFunction(
        [MOI.ScalarAffineTerm(convert(T, t.coefficient), t.variable_index) for t in f.terms],
        convert(T, f.constant)
    )
end

In [ ]:
function Base.getindex(Q::SymMatrix, i::Integer, j::Integer)
    if i <= j
        return Q.Q[MultivariateMoments.trimap(j, i)]
    else
        return conj(Q[j, i])
    end
end

In [ ]:
using SumOfSquares
using ComplexOptInterface
const COI = ComplexOptInterface
using MultivariatePolynomials
const MP = MultivariatePolynomials
function SumOfSquares.vectorization(::Type{COI.HermitianPositiveSemidefiniteConeTriangle})
    return SumOfSquares.HermitianVectorized()
end
function SumOfSquares.matrix_cone(::Type{COI.HermitianPositiveSemidefiniteConeTriangle}, d)
    return COI.HermitianPositiveSemidefiniteConeTriangle(d)
end
function add_constraint(model, p, cone; kws...)
    coefs = MOI.Utilities.vectorize(MP.coefficients(p))
    monos = MP.monomials(p)
    set = JuMP.moi_set(cone, monos; kws...)
    #MOI.Bridges.debug(model, typeof(coefs), typeof(set))
    MOI.add_constraint(model, coefs, set)
end


In [ ]:
import ComplexOptInterface
const COI = ComplexOptInterface
function SumOfSquares.matrix_constructor(::Type{COI.HermitianPositiveSemidefiniteConeTriangle}, T::Type)
    return (Q, basis) -> SumOfSquares.MultivariateMoments.VectorizedHermitianMatrix{eltype(Q), T}(Q, basis)
end
function SumOfSquares.matrix_constructor(::Type{COI.HermitianPositiveSemidefiniteConeTriangle}, ::Type{Complex{T}}) where T
    return SumOfSquares.matrix_constructor(COI.HermitianPositiveSemidefiniteConeTriangle, T)
end
function SumOfSquares.matrix_cone(::Type{COI.HermitianPositiveSemidefiniteConeTriangle}, d)
    return COI.HermitianPositiveSemidefiniteConeTriangle(d)
end

With hermition matrices

In [ ]:
const MA = SumOfSquares.MA
using MultivariatePolynomials
const MP = MultivariatePolynomials
function MA.promote_operation(::typeof(*), PT::Type{<:MP.APL{S}},
                              QT::Type{<:MP.APL{T}}) where {S, T}
    ST = MA.promote_operation(*, S, T)
    U = MA.promote_operation(+, ST, ST)
    return polynomialtype(promote_type(monomialtype(PT), monomialtype(QT)), U)
end

In [ ]:
A = CondensedMatterSOS.SpinMonomial
B = CondensedMatterSOS.SpinPolynomial{MOI.ScalarAffineFunction{Complex{Float64}}}
SumOfSquares.MA.promote_operation(*, A, B)

In [ ]:
using MosekTools
solver = optimizer_with_attributes(() -> MOIU.CachingOptimizer(MOIU.UniversalFallback(MOIU.Model{Float64}()), Mosek.Optimizer()), MOI.Silent() => true)
bound, ν = hamiltonian_energy(2, 2, solver, cone=NonnegPolyInnerCone{MOI.PositiveSemidefiniteConeTriangle}())
bound

In [ ]:
model = Model(Mosek.Optimizer)
@variable(model, a)
@variable(model, Q[1:7, 1:7] in PSDCone())
y = [Q[i, j] for j in 1:7 for i in 1:j]
x = [a; y];
@objective(model, Max, a)

In [ ]:
@constraints(model, begin
 -2x[13] == 2.0
 2x[18] == -0.0
 2x[24] == -0.0
 2x[14] == -0.0
 2x[19] == -0.0
 2x[25] == -0.0
 2x[15] == -0.0
 2x[20] == -0.0
 2x[26] == -0.0
 2x[3] == -0.0
 2x[5] == -0.0
 -2x[8] == -1.0
 2x[12] == -0.0
 2x[17] == -0.0
 -2x[23] == -1.0
 x[1] - x[2] - x[4] - x[7] - x[11] - x[16] - x[22] - x[29] == -0.0
end)

In [ ]:
optimize!(model)
MosekTools.Mosek.writedata(model.moi_backend.optimizer.model.task, "ok.cbf")

In [ ]:
print(model.moi_backend.optimizer.model)

In [ ]:
termination_status(model)

In [ ]:
ν.Q

In [ ]:
xx = nothing
SumOfSquares.SparseGramMatrix(x) = (@show typeof(x); global xx; xx = x; error())

In [ ]:
xx[2].Q

In [ ]:
using MosekTools
solver = optimizer_with_attributes(Mosek.Optimizer, MOI.Silent() => true)
bound, ν = hamiltonian_energy(2, 2, solver, sparsity = MonomialSparsity(1))
bound

In [ ]:
ν.Q

With real symmetric matrices

In [13]:
bound, ν = hamiltonian_energy(2, 2, solver, cone=NonnegPolyInnerCone{MOI.PositiveSemidefiniteConeTriangle}())
bound

monomials(variables(H), 0:1) = CondensedMatterSOS.SpinMonomial[1, σˣ₁, σʸ₁, σᶻ₁, σˣ₂, σʸ₂, σᶻ₂]
H = (-2.0 + 0.0im)σˣ₁σˣ₂ + -σᶻ₁ + -σᶻ₂
γ = x[1]
H - (1.0 + 0.0im) * γ = (-2.0 + 0.0im)σˣ₁σˣ₂ + (-1.0 + 0.0im)σᶻ₁ + (-1.0 + 0.0im)σᶻ₂ + (-x[1])


7×7 Array{MathOptInterface.SingleVariable,2}:
 x[2]   x[3]   x[5]   x[8]   x[12]  x[17]  x[23]
 x[3]   x[4]   x[6]   x[9]   x[13]  x[18]  x[24]
 x[5]   x[6]   x[7]   x[10]  x[14]  x[19]  x[25]
 x[8]   x[9]   x[10]  x[11]  x[15]  x[20]  x[26]
 x[12]  x[13]  x[14]  x[15]  x[16]  x[21]  x[27]
 x[17]  x[18]  x[19]  x[20]  x[21]  x[22]  x[28]
 x[23]  x[24]  x[25]  x[26]  x[27]  x[28]  x[29]

g = GramMatrix{MathOptInterface.SingleVariable,MonomialBasis{CondensedMatterSOS.SpinMonomial,Array{CondensedMatterSOS.SpinMonomial,1}},MathOptInterface.ScalarAffineFunction{Complex{Float64}},SymMatrix{MathOptInterface.SingleVariable}}(MathOptInterface.SingleVariable[x[2] x[3] x[5] x[8] x[12] x[17] x[23]; x[3] x[4] x[6] x[9] x[13] x[18] x[24]; x[5] x[6] x[7] x[10] x[14] x[19] x[25]; x[8] x[9] x[10] x[11] x[15] x[20] x[26]; x[12] x[13] x[14] x[15] x[16] x[21] x[27]; x[17] x[18] x[19] x[20] x[21] x[22] x[28]; x[23] x[24] x[25] x[26] x[27] x[28] x[29]], MonomialBasis{CondensedMatterSOS.SpinMonomial,Array{CondensedMatterSOS.SpinMonomial,1}}(CondensedMatterSOS.SpinMonomial[1, σˣ₁, σʸ₁, σᶻ₁, σˣ₂, σʸ₂, σᶻ₂]))
MP.monomial(t) = σˣ₁σˣ₂
MOIU.canonical(MP.coefficient(t)) = -2x[13] - 2
MP.monomial(t) = σˣ₁σʸ₂
MOIU.canonical(MP.coefficient(t)) = -2x[18]
MP.monomial(t) = σˣ₁σᶻ₂
MOIU.canonical(MP.coefficient(t)) = -2x[24]
MP.monomial(t) = σʸ₁σˣ₂
MOIU.canonical(MP.coefficient(t)) = -2x[14]
MP.monomial(t

-3.9999999997960343

In [ ]:
ν.Q

In [14]:
bound, ν = hamiltonian_energy(2, 4, solver, cone=NonnegPolyInnerCone{MOI.PositiveSemidefiniteConeTriangle}())
bound

16×16 Array{MathOptInterface.SingleVariable,2}:
 x[2]    x[3]    x[5]    x[8]    x[12]   …  x[80]   x[93]   x[107]  x[122]
 x[3]    x[4]    x[6]    x[9]    x[13]      x[81]   x[94]   x[108]  x[123]
 x[5]    x[6]    x[7]    x[10]   x[14]      x[82]   x[95]   x[109]  x[124]
 x[8]    x[9]    x[10]   x[11]   x[15]      x[83]   x[96]   x[110]  x[125]
 x[12]   x[13]   x[14]   x[15]   x[16]      x[84]   x[97]   x[111]  x[126]
 x[17]   x[18]   x[19]   x[20]   x[21]   …  x[85]   x[98]   x[112]  x[127]
 x[23]   x[24]   x[25]   x[26]   x[27]      x[86]   x[99]   x[113]  x[128]
 x[30]   x[31]   x[32]   x[33]   x[34]      x[87]   x[100]  x[114]  x[129]
 x[38]   x[39]   x[40]   x[41]   x[42]      x[88]   x[101]  x[115]  x[130]
 x[47]   x[48]   x[49]   x[50]   x[51]      x[89]   x[102]  x[116]  x[131]
 x[57]   x[58]   x[59]   x[60]   x[61]   …  x[90]   x[103]  x[117]  x[132]
 x[68]   x[69]   x[70]   x[71]   x[72]      x[91]   x[104]  x[118]  x[133]
 x[80]   x[81]   x[82]   x[83]   x[84]      x[92]   

monomials(variables(H), 0:1) = CondensedMatterSOS.SpinMonomial[1, σˣ₁, σʸ₁, σᶻ₁, σˣ₂, σʸ₂, σᶻ₂]
H = (-2.0 + 0.0im)σˣ₁σˣ₂ + -σᶻ₁ + -σᶻ₂
γ = x[1]
H - (1.0 + 0.0im) * γ = (-2.0 + 0.0im)σˣ₁σˣ₂ + (-1.0 + 0.0im)σᶻ₁ + (-1.0 + 0.0im)σᶻ₂ + (-x[1])
g = GramMatrix{MathOptInterface.SingleVariable,MonomialBasis{CondensedMatterSOS.SpinMonomial,Array{CondensedMatterSOS.SpinMonomial,1}},MathOptInterface.ScalarAffineFunction{Complex{Float64}},SymMatrix{MathOptInterface.SingleVariable}}(MathOptInterface.SingleVariable[x[2] x[3] x[5] x[8] x[12] x[17] x[23] x[30] x[38] x[47] x[57] x[68] x[80] x[93] x[107] x[122]; x[3] x[4] x[6] x[9] x[13] x[18] x[24] x[31] x[39] x[48] x[58] x[69] x[81] x[94] x[108] x[123]; x[5] x[6] x[7] x[10] x[14] x[19] x[25] x[32] x[40] x[49] x[59] x[70] x[82] x[95] x[109] x[124]; x[8] x[9] x[10] x[11] x[15] x[20] x[26] x[33] x[41] x[50] x[60] x[71] x[83] x[96] x[110] x[125]; x[12] x[13] x[14] x[15] x[16] x[21] x[27] x[34] x[42] x[51] x[61] x[72] x[84] x[97] x[111] x[126]; x[17] x[18] 

-2.8284271247461685

In [15]:
ν.Q

16×16 SymMatrix{Complex{Float64}}:
     0.324992+0.0im   1.66189e-19+0.0im  …     0.0398845+0.0im
  1.66189e-19+0.0im       0.26383+0.0im      6.37605e-17+0.0im
  5.37394e-18+0.0im  -1.64664e-12+0.0im     -4.32608e-17+0.0im
    -0.129003+0.0im  -2.37386e-17+0.0im        -0.129003+0.0im
  5.29416e-19+0.0im     -0.252115+0.0im     -6.06719e-17+0.0im
  2.25684e-18+0.0im  -4.56613e-12+0.0im  …   3.05035e-17+0.0im
    -0.129003+0.0im  -2.34377e-17+0.0im        -0.129003+0.0im
    -0.247885+0.0im   3.02053e-18+0.0im       -0.0101216+0.0im
  2.51721e-12+0.0im    2.7711e-17+0.0im     -1.17356e-11+0.0im
 -2.24666e-18+0.0im     -0.120997+0.0im     -2.75885e-17+0.0im
 -2.51721e-12+0.0im  -2.55555e-17+0.0im  …   1.17356e-11+0.0im
    0.0101216+0.0im   4.84603e-17+0.0im         0.247885+0.0im
 -4.46917e-18+0.0im  -2.23745e-12+0.0im      9.01884e-17+0.0im
 -2.68223e-18+0.0im     0.0927146+0.0im       2.4287e-17+0.0im
  1.12115e-18+0.0im   4.81084e-12+0.0im     -8.46979e-17+0.0im
    0.0398845+0.0im 

# Draft

The rest of this notebooks are drafts that should be deleted

In [ ]:
H

In [ ]:
monomials(σ[1], 1:3)

In [ ]:
import MultivariatePolynomials
const MP = MultivariatePolynomials
v = [1, σx[1], σx[2], σy[1], σy[2], σz[1], σz[2]]

In [ ]:
cone = SOSCone()
certificate = SumOfSquares.Certificate.MaxDegree(cone, MonomialBasis, 2)
Certificate.sparsity(H, MonomialSparsity(0), certificate)

min_σ H(σ) = -2√2

max γ s.t. H >= γ -> H - γ = v' Q v

In [ ]:
function _mat(x::Vector, n)
    Q = Matrix{eltype(x)}(undef, n, n)
    k = 0
    for j in 1:n
        for i in 1:j
            k += 1
            Q[i, j] = Q[j, i] = x[k]
        end
    end
    return Q
end

In [ ]:
function _mapcoef(op, f::MOI.ScalarAffineFunction)
    MOI.ScalarAffineFunction([
        MOI.ScalarAffineTerm(op(t.coefficient), t.variable_index) for t in f.terms
    ], op(f.constant))
end

# Real

In [ ]:
function operate!(::typeof(*), ::Type{T}, α::T,
                  f::MOI.ScalarAffineFunction) where T
    map_terms!(term -> operate_term(*, term, α), f)
    rmul!(f.constants, α)
    return f
end
function operate(::typeof(*), ::Type{T}, α::T, f::TypedLike{T}) where T
    return operate!(*, T, copy(f), α)
end

In [ ]:
using SumOfSquares
Base.promote_rule(::Type{T}, ::Type{MOI.SingleVariable}) where {T<:Number} = MOI.ScalarAffineFunction{T}
PolyJuMP.non_constant(a::Vector{<:MOI.AbstractFunction}) = a
function MultivariatePolynomials.variables(monos::Vector{CondensedMatterSOS.SpinMonomial})
    vars = Set{CondensedMatterSOS.SpinVariable}()
    for mono in monos
        union!(vars, variables(mono))
    end
    return sort(collect(vars), rev=true)
end
function Base.convert(::Type{MOI.ScalarAffineFunction{T}}, f::MOI.ScalarAffineFunction) where T
    return MOI.ScalarAffineFunction(
        [MOI.ScalarAffineTerm(convert(T, t.coefficient), t.variable_index) for t in f.terms],
        convert(T, f.constant)
    )
end
function MOI.Utilities.operate(::typeof(*), ::Type{Complex{Float64}}, a::Complex{Float64}, f::MOI.ScalarAffineFunction{Float64})
    MOI.Utilities.operate(*, Complex{Float64}, a, convert(MOI.ScalarAffineFunction{Complex{Float64}}, f))
end
MultivariatePolynomials.variables(p::CondensedMatterSOS.SpinPolynomial) = variables(monomials(p))
Base.copy(t::CondensedMatterSOS.SpinTerm) = CondensedMatterSOS.SpinTerm(t.coefficient, t.monomial)
Base.:*(f::MOI.ScalarAffineFunction{Complex{Float64}}, a::Number) = f * convert(Complex{Float64}, a)
Base.:*(f::MOI.ScalarAffineFunction{Complex{Float64}}, a::Complex{Float64}) = MOI.Utilities.operate(*, Complex{Float64}, a, f)
Base.:+(a::Int, f::MOI.ScalarAffineFunction{Complex{Float64}}) = convert(Complex{Float64}, a) + f
Base.:+(a::Complex{Float64}, f::MOI.ScalarAffineFunction{Complex{Float64}}) = MOI.Utilities.operate(+, Complex{Float64}, a, f)

In [ ]:
function add_constraint(model, p, cone; kws...)
    coefs = MOI.Utilities.vectorize(PolyJuMP.non_constant_coefficients(p))
    monos = MP.monomials(p)
    set = JuMP.moi_set(cone, monos; kws...)
    MOI.add_constraint(model, coefs, set)
end

In [ ]:
using SumOfSquares
using ComplexOptInterface
const COI = ComplexOptInterface
using MosekTools
solver = optimizer_with_attributes(Mosek.Optimizer, MOI.Silent() => true)
optimizer = MOI.instantiate(solver, with_bridge_type=Float64)
model = MOI.Bridges.Constraint.SingleBridgeOptimizer{SumOfSquares.Bridges.Constraint.SOSPolynomialBridge{Complex{Float64}}}(
    MOI.Bridges.Constraint.SingleBridgeOptimizer{PolyJuMP.ZeroPolynomialBridge{Complex{Float64}}}(
    MOI.Bridges.Constraint.SingleBridgeOptimizer{COI.Bridges.Constraint.SplitZeroBridge{Float64}}(
    optimizer
)))
γ = MOI.SingleVariable(MOI.add_variable(model))
add_constraint(model, H - γ, SOSCone(), ideal_certificate=SumOfSquares.Certificate.MaxDegree(SOSCone(), MonomialBasis, 2))
MOI.set(model, MOI.ObjectiveSense(), MOI.MAX_SENSE)
MOI.set(model, MOI.ObjectiveFunction{MOI.SingleVariable}(), γ)
MOI.optimize!(model)
MOI.get(model, MOI.ObjectiveValue())

In [ ]:
function energy(H, maxdegree, cone)
    solver = optimizer_with_attributes(Mosek.Optimizer, MOI.Silent() => true)
    optimizer = MOI.instantiate(solver, with_bridge_type=Float64)
    model = MOI.Bridges.Constraint.SingleBridgeOptimizer{SumOfSquares.Bridges.Constraint.SOSPolynomialBridge{Complex{Float64}}}(
        MOI.Bridges.Constraint.SingleBridgeOptimizer{PolyJuMP.ZeroPolynomialBridge{Complex{Float64}}}(
        MOI.Bridges.Variable.SingleBridgeOptimizer{COI.Bridges.Variable.HermitianToSymmetricPSDBridge{Float64}}(
        MOI.Bridges.Constraint.SingleBridgeOptimizer{COI.Bridges.Constraint.SplitZeroBridge{Float64}}(
        optimizer
    ))))
    γ = MOI.SingleVariable(MOI.add_variable(model))
    cert = SumOfSquares.Certificate.MaxDegree(cone, MonomialBasis, maxdegree)
    add_constraint(model, H - γ, SOSCone(), ideal_certificate=cert)
    MOI.set(model, MOI.ObjectiveSense(), MOI.MAX_SENSE)
    MOI.set(model, MOI.ObjectiveFunction{MOI.SingleVariable}(), γ)
    MOI.optimize!(model)
    if MOI.get(model, MOI.TerminationStatus()) != MOI.OPTIMAL
        @warn("Termination status: $(MOI.get(model, MOI.TerminationStatus())), $(MOI.get(model, MOI.RawStatusString()))")
    end
    MOI.get(model, MOI.ObjectiveValue())
end

In [ ]:
energy(H, 2, NonnegPolyInnerCone{MOI.PositiveSemidefiniteConeTriangle}())

In [ ]:
energy(H, 4, NonnegPolyInnerCone{MOI.PositiveSemidefiniteConeTriangle}())

In [ ]:
function SumOfSquares.matrix_cone(::Type{COI.HermitianPositiveSemidefiniteConeTriangle}, d)
    return COI.HermitianPositiveSemidefiniteConeTriangle(d)
end
using MutableArithmetics
const MA = MutableArithmetics
function MOI.Utilities._add_sub_affine_terms(
    op::Union{typeof(+), typeof(-)}, terms::Vector{MOI.ScalarAffineTerm{T}},
    α::T, f::MOI.SingleVariable) where T
    push!(terms, MOI.ScalarAffineTerm(op(α), f.variable))
end
MOI.Utilities._constant(::Type{T}, ::MOI.SingleVariable) where T = zero(T)
function SumOfSquares.add_gram_matrix(
        model::MOI.ModelLike,
        matrix_cone_type::Type{COI.HermitianPositiveSemidefiniteConeTriangle},
        basis::AbstractPolynomialBasis, T::Type{<:Complex})
    n = length(basis)
    Q, cQ = MOI.add_constrained_variables(model, SumOfSquares.matrix_cone(matrix_cone_type, n))
    N = MOI.dimension(MOI.PositiveSemidefiniteConeTriangle(n))
    C = [zero(MOI.ScalarAffineFunction{T}) for i in 1:N]
    k_real = 0
    k_imag = 0
    for j in 1:n
        for i in 1:j
            k_real += 1
            MA.mutable_operate!(MA.add_mul, C[k_real], one(T), MOI.SingleVariable(Q[k_real]))
            if i != j
                k_imag += 1
                MA.mutable_operate!(MA.add_mul, C[k_real], one(T) * im, MOI.SingleVariable(Q[N + k_imag]))
            end
        end
    end
    q = SumOfSquares.build_gram_matrix(C, basis, T)
    return q, Q, cQ
end

In [ ]:
q = SumOfSquares.SymMatrix([1.0, 1.0 + im, 2.0], 2)
function Base.getindex(Q::SymMatrix, i::Integer, j::Integer)
    if i <= j
        return Q.Q[MultivariateMoments.trimap(j, i)]
    else
        return conj(Q[j, i])
    end
end
function COI.Bridges.Constraint.operate_coefficients(f, T::Type, func::MOI.ScalarAffineFunction)
    return MOI.ScalarAffineFunction(
        [COI.Bridges.Constraint.operate_coefficient(f, T, term) for term in func.terms],
        f(func.constant)
    )
end
# We assume MOI variables are real
Base.conj(func::MOI.Utilities.TypedLike{T}) where T = COI.Bridges.Constraint.operate_coefficients(conj, T, func)
q

In [ ]:
default_name(vi) = string("x[", vi.value, "]")
function function_string(::Type{JuMP.REPLMode}, v::MOI.VariableIndex, name = default_name)
    var_name = name(v)
    if !isempty(var_name)
        return var_name
    else
        return "noname"
    end
end
function function_string(::Type{JuMP.IJuliaMode}, v::MOI.VariableIndex, name = default_name)
    var_name = name(v)
    if !isempty(var_name)
        # TODO: This is wrong if variable name constains extra "]"
        return replace(replace(var_name, "[" => "_{", count = 1), "]" => "}")
    else
        return "noname"
    end
end
# Whether something is zero or not for the purposes of printing it
# oneunit is useful e.g. if coef is a Unitful quantity. The second `abs` is import if it is complex.
_is_zero_for_printing(coef) = abs(coef) < 1e-10 * abs(oneunit(coef))
# Whether something is one or not for the purposes of printing it.
_is_one_for_printing(coef) = _is_zero_for_printing(abs(coef) - oneunit(coef))
_is_one_for_printing(coef::Complex) = _is_one_for_printing(real(coef)) && _is_zero_for_printing(imag(coef))
_sign_string(coef) = coef < zero(coef) ? " - " : " + "
_sign_string(coef) = coef < zero(coef) ? " - " : " + "
function function_string(mode, func::MOI.ScalarAffineFunction, show_constant=true)
    # If the expression is empty, return the constant (or 0)
    if isempty(func.terms)
        return show_constant ? JuMP._string_round(MOI.constant(func)) : "0"
    end

    term_str = Vector{String}(undef, 2length(func.terms))
    elm = 1

    for term in func.terms
        pre = _is_one_for_printing(term.coefficient) ? "" : JuMP._string_round(term.coefficient) * " "

        term_str[2 * elm - 1] = "+"
        term_str[2 * elm] = string(pre, function_string(mode, term.variable_index))
        elm += 1
    end

    if elm == 1
        # Will happen with cancellation of all terms
        # We should just return the constant, if its desired
        return show_constant ? JuMP._string_round(a.constant) : "0"
    else
        # Correction for very first term - don't want a " + "/" - "
        term_str[1] = (term_str[1] == " - ") ? "-" : ""
        ret = join(term_str[1 : 2 * (elm - 1)])
        if !_is_zero_for_printing(MOI.constant(func)) && show_constant
            ret = string(ret, "+",
                         JuMP._string_round(MOI.constant(constant)))
        end
        return ret
    end
end
function Base.show(io::IO, f::MOI.AbstractScalarFunction)
    print(io, function_string(JuMP.REPLMode, f))
end
function Base.show(io::IO, ::MIME"text/latex", f::MOI.AbstractScalarFunction)
    print(io, JuMP._wrap_in_math_mode(function_string(JuMP.IJuliaMode, f)))
end

In [ ]:
x = MOI.SingleVariable(MOI.VariableIndex(1))
conj((im) * x)

In [ ]:
energy(H, 2, NonnegPolyInnerCone{COI.HermitianPositiveSemidefiniteConeTriangle}())

In [ ]:
using MosekTools
optimizer = Mosek.Optimizer()
MOI.set(optimizer, MOI.Silent(), true)
n = length(v)
x, cx = MOI.add_constrained_variables(optimizer, MOI.PositiveSemidefiniteConeTriangle(n))
γ = MOI.SingleVariable(MOI.add_variable(optimizer))
F = MOI.ScalarAffineFunction{Complex{Float64}}
Q = _mat(F.(MOI.SingleVariable.(x)), n)
gram = sum(Q[i, j] * (1.0 * monos[i] * monos[j]) for i in 1:n for j in 1:n)
z = H - convert(F, γ) - gram
for c in MP.coefficients(z)
    for a in [_mapcoef(real, c), _mapcoef(imag, c)]
        MOI.Utilities.normalize_and_add_constraint(optimizer, a, MOI.EqualTo(0.0))
    end
end
MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MAX_SENSE)
MOI.set(optimizer, MOI.ObjectiveFunction{MOI.SingleVariable}(), γ)
MOI.optimize!(optimizer)
MOI.get(optimizer, MOI.ObjectiveValue())

# Complex

In [ ]:
using MosekTools
import MultivariatePolynomials
const MP = MultivariatePolynomials
optimizer = MOI.instantiate(Mosek.Optimizer, with_bridge_type=Float64)
monos = [1, σx[1], σx[2], σy[1], σy[2], σz[1], σz[2]]
n = length(monos)
x, cx = MOI.add_constrained_variables(optimizer, MOI.PositiveSemidefiniteConeTriangle(2n))
X = _mat(MOI.SingleVariable.(x), 2n)
X11 = X[1:n, 1:n]
X22 = X[n .+ (1:n), n .+ (1:n)]
X12 = X[1:n, n .+ (1:n)]
for j in 1:n
    for i in 1:j
        MOI.add_constraint(optimizer, MOI.Utilities.operate(-, Float64, X11[i, j], X22[i, j]), MOI.EqualTo(0.0))
        if i == j
            MOI.add_constraint(optimizer, convert(MOI.ScalarAffineFunction{Float64}, X12[i, j]), MOI.EqualTo(0.0))
        else
            MOI.add_constraint(optimizer, MOI.Utilities.operate(+, Float64, X12[i, j], X12[j, i]), MOI.EqualTo(0.0))
        end
    end
end
γ = MOI.SingleVariable(MOI.add_variable(optimizer))
F = MOI.ScalarAffineFunction{Complex{Float64}}
Q_real = F.(X11)
Q_comp = F.(X12)
Q = Q_real + (-1.0*im) * Q_comp
gram = sum(Q[i, j] * (1.0 * monos[i] * monos[j]) for i in 1:n for j in 1:n)
z = H - convert(F, γ) - gram
duals = Dict()
for term in MP.terms(z)
    duals[MP.monomial(term)] = map([real, imag]) do f
        a = _mapcoef(f, MP.coefficient(term))
        MOI.Utilities.normalize_and_add_constraint(optimizer, a, MOI.EqualTo(0.0))
    end
end
MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MAX_SENSE)
MOI.set(optimizer, MOI.ObjectiveFunction{MOI.SingleVariable}(), γ)
MOI.optimize!(optimizer)
@show MOI.get(optimizer, MOI.TerminationStatus())
MOI.get(optimizer, MOI.ObjectiveValue())

In [ ]:
using MosekTools
optimizer = MOI.instantiate(Mosek.Optimizer, with_bridge_type=Float64)
n = length(v)
x, cx = MOI.add_constrained_variables(optimizer, MOI.PositiveSemidefiniteConeTriangle(2n))
X = _mat(MOI.SingleVariable.(x), 2n)
X11 = X[1:n, 1:n]
X22 = X[n .+ (1:n), n .+ (1:n)]
X12 = X[1:n, n .+ (1:n)]
for j in 1:n
    for i in 1:j
        MOI.add_constraint(optimizer, MOI.Utilities.operate(-, Float64, X11[i, j], X22[i, j]), MOI.EqualTo(0.0))
        if i == j
            MOI.add_constraint(optimizer, convert(MOI.ScalarAffineFunction{Float64}, X12[i, j]), MOI.EqualTo(0.0))
        else
            MOI.add_constraint(optimizer, MOI.Utilities.operate(+, Float64, X12[i, j], X12[j, i]), MOI.EqualTo(0.0))
        end
    end
end
γ = MOI.SingleVariable(MOI.add_variable(optimizer))
F = MOI.ScalarAffineFunction{Complex{Float64}}
Q_real = F.(X11)
Q_comp = F.(X12)
Q = Q_real + (-1.0*im) * Q_comp
gram = sum(Q[i, j] * (1.0 * monos[i] * monos[j]) for i in 1:n for j in 1:n)
z = H - convert(F, γ) - gram
duals = Dict()
for term in MP.terms(z)
    duals[MP.monomial(term)] = map([real, imag]) do f
        a = _mapcoef(f, MP.coefficient(term))
        MOI.Utilities.normalize_and_add_constraint(optimizer, a, MOI.EqualTo(0.0))
    end
end
MOI.set(optimizer, MOI.ObjectiveSense(), MOI.MAX_SENSE)
MOI.set(optimizer, MOI.ObjectiveFunction{MOI.SingleVariable}(), γ)
MOI.optimize!(optimizer)
@show MOI.get(optimizer, MOI.TerminationStatus())

In [ ]:
@show MOI.get(optimizer, MOI.TerminationStatus())

In [ ]:
-2 * √2

In [ ]:
MOI.get(optimizer, MOI.ObjectiveValue())

In [ ]:
MOI.get(optimizer, MOI.SolveTime())

In [ ]:
Mfull = round.(_mat(MOI.get(optimizer, MOI.ConstraintDual(), cx), 2n), digits=6)
M11 = Mfull[1:n, 1:n]
M12 = Mfull[1:n, n .+ (1:n)]
M12 + im * M12

In [ ]:
Mfull = round.(_mat(MOI.get(optimizer, MOI.ConstraintDual(), cx), 2n), digits=6)
M11 = Mfull[1:n, 1:n]
M12 = Mfull[1:n, n .+ (1:n)]
M12 + im * M12

In [ ]:
using MultivariateMoments
function _moment_matrix(μ::MultivariateMoments.Measure{T}, X) where T
    function getmom(i, j)
        x = X[i] * X[j]
        for m in moments(μ)
            if monomial(m) == monomial(x)
                return MP.coefficient(x) * moment_value(m)
            end
        end
        throw(ArgumentError("μ does not have the moment $(x)"))
    end
    return MultivariateMoments.MomentMatrix{T}(getmom, X)
end

In [ ]:
a = Complex{Float64}[]
m = MP.monomialtype(H)[]
for (key, value) in duals
    push!(m, key)
    real_dual = MOI.get(optimizer, MOI.ConstraintDual(), value[1])
    comp_dual = MOI.get(optimizer, MOI.ConstraintDual(), value[2])
    push!(a, real_dual + im * comp_dual)
end
μ = measure(a, m)
ν = _moment_matrix(μ, v)

In [ ]:
round.(ν.Q, digits = 1)

In [ ]:
using LinearAlgebra
Γ = Hermitian([
    1 0 0   0     0     1/2 1/2
    0 1 1/2 1/2im 0     0   0
    0 0 1   0     1/2im 0   0
    0 0 0   1    -1/2   0   0
    0 0 0   0     1     0   0
    0 0 0   0     0     1   1
    0 0 0   0     0     0   1
])